In [1]:
import cv2
import dlib
import PIL.Image
import numpy as np
from imutils import face_utils
import argparse
from pathlib import Path
import os
import ntpath

In [2]:
# Chargement des modèles pré-entraînés 
# Remplacez les chemins absolus vers vos fichiers .dat ici
pose_predictor_68_point = dlib.shape_predictor("/home/la-mus/AI/projet/pretrained_model/shape_predictor_68_face_landmarks.dat")
pose_predictor_5_point = dlib.shape_predictor("/home/la-mus/AI/projet/pretrained_model/shape_predictor_5_face_landmarks.dat")
face_encoder = dlib.face_recognition_model_v1("/home/la-mus/AI/projet/pretrained_model/dlib_face_recognition_resnet_model_v1.dat")
# Détecteur de visages frontal
face_detector = dlib.get_frontal_face_detector()

# Fonction de transformation des coordonnées 
def transform(image, face_locations):
    """
    Convertit les rectangles de détection de visage dlib (top, right, bottom, left) 
    en coordonnées gérées pour OpenCV et les limites de l'image.
    
    Args :
        image (numpy array) : Image source.
        face_locations (list) : Liste d'objets rectangles dlib délimitant les visages.

    Retour :
        coord_faces (list) : Liste de tuples coordonnées corrigées (top, right, bottom, left).
    """
    coord_faces = []
    for face in face_locations:
        rect = face.top(), face.right(), face.bottom(), face.left()
        coord_face = (
            max(rect[0], 0),                    # top limité à 0 minimum
            min(rect[1], image.shape[1]),      # right limité aux bords de l'image
            min(rect[2], image.shape[0]),      # bottom limité aux bords
            max(rect[3], 0)                    # left limité à 0 minimum
        )
        coord_faces.append(coord_face)
    return coord_faces

# Fonction d'encodage des visages 
def encode_face(image):
    """
    Détecte les visages dans l'image, calcule leurs encodages et points de repère (landmarks).
    
    Args :
        image (numpy array) : Image RGB.

    Retour :
        face_encodings_list (list) : Liste des encodages des visages détectés.
        face_locations (list) : Coordonnées des visages détectés.
        landmarks_list (list) : Liste des points de repère pour chaque visage.
    """
    face_locations = face_detector(image, 1)  # Détection visage avec dlib

    face_encodings_list = []
    landmarks_list = []

    for face_location in face_locations:
        # Extraction des points de repère (shape/landmarks)
        shape = pose_predictor_68_point(image, face_location)

        # Calcul encoding du visage (vecteur de caractéristiques)
        face_encodings_list.append(
            np.array(face_encoder.compute_face_descriptor(image, shape, num_jitters=1))
        )
        # Conversion shape en tableau numpy pour landmarks
        shape_np = face_utils.shape_to_np(shape)
        landmarks_list.append(shape_np)

    # Transformation des coordonnées pour OpenCV
    face_locations = transform(image, face_locations)

    return face_encodings_list, face_locations, landmarks_list

# Fonction principale de reconnaissance 
def easy_face_reco(frame, known_face_encodings, known_face_names):
    """
    Analyse une image/frame vidéo, détecte et reconnaît les visages connus.
    Dessine les boîtes et noms autour des visages détectés.

    Args :
        frame (numpy array) : Image BGR (vidéo frame).
        known_face_encodings (list) : Liste des encodages des visages connus.
        known_face_names (list) : Liste des noms associés aux visages connus.
    """
    rgb_small_frame = frame[:, :, ::-1]  # Conversion BGR (cv2) -> RGB (dlib)

    # Encodage des visages détectés dans la frame
    face_encodings_list, face_locations_list, landmarks_list = encode_face(rgb_small_frame)

    face_names = []

    for face_encoding in face_encodings_list:
        if len(face_encoding) == 0:
            return np.empty((0))

        # Calcul distance entre visages détectés et connus
        vectors = np.linalg.norm(known_face_encodings - face_encoding, axis=1)

        tolerance = 0.6

        result = [vector <= tolerance for vector in vectors]

        if True in result:
            first_match_index = result.index(True)
            name = known_face_names[first_match_index]
        else:
            name = "Unknown"
        face_names.append(name)

    # Dessin des rectangles et noms autour des visages détectés
    for (top, right, bottom, left), name in zip(face_locations_list, face_names):
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        cv2.rectangle(frame, (left, bottom - 30), (right, bottom), (0, 255, 0), cv2.FILLED)
        cv2.putText(frame, name, (left + 2, bottom - 2), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 1)

    # Dessin des points de repère (landmarks) sur les visages
    for shape in landmarks_list:
        for (x, y) in shape:
            cv2.circle(frame, (x, y), 1, (255, 0, 255), -1)


# ---------------------------------------------------------------
# ------------ Classe pour simuler les arguments dans notebook --
# ---------------------------------------------------------------
class Args:
    # Chemin vers dossier contenant les images des visages connus
    input = "/home/la-mus/AI/projet/known_face"


# ---------------------------------------------------------------
# --------------------- Code principal --------------------------
# ---------------------------------------------------------------
if __name__ == '__main__':

    args = Args()

    print('[INFO] Importing faces...')

    # Recherche tous les fichiers images dans le dossier spécifié (jpg et png)
    face_to_encode_path = Path(args.input)
    files = [file_ for file_ in face_to_encode_path.rglob('*.jpg')]

    for file_ in face_to_encode_path.rglob('*.png'):
        files.append(file_)

    if len(files) == 0:
        raise ValueError(f'No faces detected in the directory: {face_to_encode_path}')

    # Extraction des noms à partir des noms de fichiers (sans extension)
    known_face_names = [os.path.splitext(ntpath.basename(file_))[0] for file_ in files]

    # Encodage de chaque visage connu dans une liste
    known_face_encodings = []
    for file_ in files:
        image = PIL.Image.open(file_)
        image = np.array(image)
        face_encoded = encode_face(image)[0][0]  # On prend le 1er visage détecté par image
        known_face_encodings.append(face_encoded)

    print('[INFO] Faces well imported')
    print('[INFO] Starting Webcam...')

    # Ouverture de la webcam
    video_capture = cv2.VideoCapture(0)
    print('[INFO] Webcam well started')
    print('[INFO] Detecting...')

    # Boucle principale : lecture et traitement des frames webcam
    while True:
        ret, frame = video_capture.read()
        if not ret:
            print("[ERROR] Cannot grab frame from webcam")
            break

        easy_face_reco(frame, np.array(known_face_encodings), known_face_names)

        # Affiche la frame annotée
        cv2.imshow('Easy Facial Recognition App', frame)

        # Quitte la boucle si on appuie sur la touche 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    print('[INFO] Stopping System')

    # Libération de la webcam et destruction des fenêtres
    video_capture.release()
    cv2.destroyAllWindows()

[INFO] Importing faces...
[INFO] Faces well imported
[INFO] Starting Webcam...
[INFO] Webcam well started
[INFO] Detecting...


TypeError: compute_face_descriptor(): incompatible function arguments. The following argument types are supported:
    1. (self: _dlib_pybind11.face_recognition_model_v1, img: numpy.ndarray[(rows,cols,3),numpy.uint8], face: _dlib_pybind11.full_object_detection, num_jitters: int = 0, padding: float = 0.25) -> _dlib_pybind11.vector
    2. (self: _dlib_pybind11.face_recognition_model_v1, img: numpy.ndarray[(rows,cols,3),numpy.uint8], num_jitters: int = 0) -> _dlib_pybind11.vector
    3. (self: _dlib_pybind11.face_recognition_model_v1, img: numpy.ndarray[(rows,cols,3),numpy.uint8], faces: _dlib_pybind11.full_object_detections, num_jitters: int = 0, padding: float = 0.25) -> _dlib_pybind11.vectors
    4. (self: _dlib_pybind11.face_recognition_model_v1, batch_img: list[numpy.ndarray[(rows,cols,3),numpy.uint8]], batch_faces: list[_dlib_pybind11.full_object_detections], num_jitters: int = 0, padding: float = 0.25) -> _dlib_pybind11.vectorss
    5. (self: _dlib_pybind11.face_recognition_model_v